# Explore the results of the experiments on the SRBench ground-truth datasets
In this notebook we show hoow to analyze the results from the benchmark at the example of the experiments without noise. For the visualizations see results/plots/srbench_groundtruth_results.ipynb

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os 
from sympy import nsimplify
import sympy

In [2]:
results_directory = r'results'

In [3]:
dataset_names = pd.read_csv('datasets', names=['Index', 'Name', 'Formula'])
dataset_names_dict = {dataset_names['Name'][i]: dataset_names['Index'][i] for i in range(len(dataset_names))}

### Without noise

In [39]:
# Loading
path = os.path.join(results_directory, 'Without noise')
results = pd.read_csv(os.path.join(path, 'results.csv'))

In [40]:
results.columns

Index(['Unnamed: 0', 'Index', 'max_deg_input', 'max_deg_output',
       'max_deg_input_denominator', 'max_deg_output_denominator',
       'max_deg_output_polynomials_specific',
       'max_deg_output_polynomials_denominator_specific', 'width',
       'function_names', 'maximal_potence', 'maximal_n_functions', 'maxiter1',
       'maxiter2', 'target_noise', 'feature_noise', 'optimizer',
       'pruning_iterations', 'pruning_cut_off', 'classifier',
       'local_minimizer', 'maxiter_per_dim_local_minimizer',
       'max_dataset_length', 'lambda_1', 'repetitions', 'parallel',
       'n_processes', 'lambda_1_cut', 'lambda_1_piecewise', 'device',
       'accuracy', 'time_limit', 'evaluations_limit', 'iterative_finetuning',
       'max_n_active_parameters', 'dataset', 'target_formula',
       'estimated_formula', 'relative_l2_train', 'relative_l2_val',
       'relative_l2_test', 'r_squared_val', 'r_squared_test', 'success',
       'training_time', 'n_active_coefficients',
       'n_active_coe

#### How to go about the finetuning results?

In [41]:
results['r_squared_test_final'] = results['r_squared_test']
results['n_active_coefficients_final'] = results['n_active_coefficients']
results['estimated_formula_final'] = results['estimated_formula']

In [42]:
# Logic: if the finetuned result is better or it is slightly worse but considerably smaller, we will take it. 
# Otherwise we take the results from the full model.
better_performance = results['r_squared_val'] < results['r_squared_val_reduced']
slightly_worse_performance = (1 - results['r_squared_val']) * 5 > (1 - results['r_squared_val_reduced'])
considerably_less_complexity = results['n_active_coefficients_reduced'] < results['n_active_coefficients'] * 0.75
sum(better_performance), sum(slightly_worse_performance), sum(considerably_less_complexity)

(78, 123, 128)

In [43]:
take_reduced_model = better_performance & (slightly_worse_performance | considerably_less_complexity)
results['r_squared_test_final'][take_reduced_model] = results['r_squared_test_reduced'][take_reduced_model]
results['n_active_coefficients_final'][take_reduced_model] = results['n_active_coefficients_reduced'][take_reduced_model]
results['estimated_formula_final'][take_reduced_model] = results['best_formula_reduced'][take_reduced_model]

/tmp/ipykernel_20350/2960987597.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['r_squared_test_final'][take_reduced_model] = results['r_squared_test_reduced'][take_reduced_model]
/tmp/ipykernel_20350/2960987597.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['n_active_coefficients_final'][take_reduced_model] = results['n_active_coefficients_reduced'][take_reduced_model]
/tmp/ipykernel_20350/2960987597.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

#### Detemine the symbolic recovery rate

In [44]:
# Go through the formulas who have an R² greater than 0.9999999999, as these are the candidates for which ParFam might have found the correct formula
# and then inspect those formulas by hand
candidates = (results['r_squared_test_final'] > 0.9999999999).values #results['r_squared_test_final'] > 0.9999999999

for i in range(len(results[candidates])):
    estimated_expression = sympy.simplify(results[candidates]['estimated_formula_final'].iloc[i],rational=False)
    print(f'i: {i}; target expression: {results[candidates]["target_formula"].iloc[i]};' 
          f'estimated expression: {estimated_expression}')
    #nsimplify(expression,tolerance=0.1,rational=False)

i: 0; target expression: mom*sqrt(Bx**2+By**2+Bz**2);estimated expression: 0.999373866478407*x0*sqrt(Abs(x1**2 + x2**2 + x3**2))
i: 1; target expression: n*(h/(2*pi));estimated expression: 0.159*x0*x1
i: 2; target expression: 2*E_n*d**2*k/(h/(2*pi));estimated expression: 12.566*x0*x1**2*x2/x3
i: 3; target expression: 2*U*(1-cos(k*d));estimated expression: 2.0*x0*(1 - cos(1.0*x1*x2 - 6.283))
i: 4; target expression: 2*pi*alpha/(n*d);estimated expression: 6.283*x0/(x1*x2)
i: 5; target expression: beta*(1+alpha*cos(theta));estimated expression: 1.0*x0*(x1*cos(1.0*x2) + 1)
i: 6; target expression: -rho_c_0*q*A_vec/m;estimated expression: -1.0*x0*x1*x2/x3
i: 7; target expression: 2*mom*B/(h/(2*pi));estimated expression: 12.566*x0*x1/x2
i: 8; target expression: sin(E_n*t/(h/(2*pi)))**2;estimated expression: 0.5*cos(12.566*x0*x1/x2 - 3.142) + 0.5
i: 9; target expression: sigma_den/epsilon*1/(1+chi);estimated expression: 1.0*x0/(x1*(x2 + 1))
i: 10; target expression: n_rho*p_d**2*Ef/(3*kb*T);e

In [45]:
failed_candidates = [] # All formulas of the candidates were correct

In [46]:
results['symbolic_recovery_final'] = candidates
index = 0
for i in range(len(results)):
    if candidates[i]:
        if index in failed_candidates:
            results['symbolic_recovery_final'].iloc[i] = False
        index += 1

In [47]:
sum(candidates) / len(candidates), results['symbolic_recovery_final'].mean()

(0.556390977443609, 0.556390977443609)

In [48]:
results['accuracy_solution_final'] = results['r_squared_test_final'] > 0.999

#### Complexity

In [ ]:
# Biggest problem: we count scalar multiplication as 2 complexity, while SRBench counts it as 1
# Small problem (which is maybe just a problem of our method: we count in a_1*\sqrt(a_2*x) 2 scalar multiplication, while it could be just 1. 

function_names = ['sin', 'cos', 'sqrt', 'exp']
variable_names = [f'x{i}' for i in range(10)]
operations = ['+', '-', '/', '*']
# formula = '(x0*sqrt(((((x2*x2)+(x1*x1)))+(x3*x3))))+sqrt(((((x2*x2)+(x1*x1)))+(x3*x3)))'
formula = '1.0*x0*(x1*cos(1.0*x2 + 18.85) + 1)'

def get_expr(formula, function_names):
    expr = None
    for function_name in function_names:
        index_start = formula.find(function_name)
        if index_start != -1:
            found_end = False
            index_stop = index_start
            while not found_end:
                index_stop = formula.find(')', index_stop+1)
                expr = formula[index_start:index_stop+1]
                if expr.count('(') == expr.count(')'):
                    found_end = True
    return expr

def is_scalar_multiplication(formula, index):
    digits = []
    for i in range(10):
        digits.append(f'{i}')
    # We assume here that there are no more than 10 variables and no potence higher than 9
    return (formula[index-1] in digits) and (formula[index-2] in digits or formula[index-2] == '.')

def count(formula, variable_names, operations):
    complexity = 0
    for variable_name in variable_names:
        complexity += formula.count(variable_name)
    # Before we get to multiplication (*) we have to remove the potences (**)
    complexity += formula.count('**')
    formula = formula.replace('**', '')
    for operation in operations:
        complexity += formula.count(operation)
    complexity += formula.count('^')
    complexity += formula.count('.')
    # Get rid of the complexity 2 for scalar multiplication
    start_index = 0
    remaining_multiplications = True
    while remaining_multiplications:
        index = formula.find('*', start_index)
        if index != -1:
            if is_scalar_multiplication(formula, index):
                complexity -= 1
            start_index = index + 1
        else:
            remaining_multiplications = False
    return complexity

def get_complexity(formula, function_names, variable_names, operations):
    expr = get_expr(formula, function_names)
    if not expr is None:
        formula_red = formula.replace(expr, variable_names[0])
    else:
        formula_red = formula
    complexity = count(formula_red, variable_names, operations)
    if not expr is None:
        complexity += count(expr, variable_names, operations)
    return complexity

In [49]:
# Compute the complexity of a formula
results['complexity_final'] = results['estimated_formula_final'].apply(get_complexity, args=(function_names, variable_names, operations))

#### Complexity

In [50]:
results.to_csv(os.path.join(path, 'results_analyzed.csv'))

In [54]:
# Loading
path = os.path.join(results_directory, 'With noise')
results = pd.read_csv(os.path.join(path, 'results_combined_analyzed.csv'))

In [55]:
results

,Index,max_deg_input,max_deg_output,max_deg_input_denominator,max_deg_output_denominator,max_deg_output_polynomials_specific,max_deg_output_polynomials_denominator_specific,width,function_names,maximal_potence,...,r_squared_val_reduced,best_formula_reduced,n_evaluations,r_squared_test_final,n_active_coefficients_final,estimated_formula_final,symbolic_recovery_final,accuracy_solution_final,simplified_formulas,complexity_final
0,120,2,4,2,3,"[1, 1, 1]","[1, 1, 1]",1,"['sqrt', 'cos', 'exp']",3,...,0.999392,1.78*x0*sqrt(Abs(0.315*x1**2 + 0.315*x2**2 + 0...,862863,1.000000,4,1.78*x0*sqrt(Abs(0.315*x1**2 + 0.315*x2**2 + 0...,True,True,1.78*x0*sqrt(Abs(0.315*x1**2 + 0.315*x2**2 + 0...,15
1,121,2,4,2,3,"[1, 1, 1]","[1, 1, 1]",1,"['sqrt', 'cos', 'exp']",3,...,0.999582,0.159*x0*x1,999766,1.000000,18,0.159*x0*x1,True,True,0.159*x0*x1,4
2,122,2,4,2,3,"[1, 1, 1]","[1, 1, 1]",1,"['sqrt', 'cos', 'exp']",3,...,0.999834,(0.042*x0**2*x1*x2 - 0.068*x0**2*x1*x3 + 0.014...,16092,1.000000,23,(0.042*x0**2*x1*x2 - 0.068*x0**2*x1*x3 + 0.014...,False,True,(0.042*x0**2*x1*x2 - 0.068*x0**2*x1*x3 + 0.014...,120
3,123,2,4,2,3,"[1, 1, 1]","[1, 1, 1]",1,"['sqrt', 'cos', 'exp']",3,...,0.999799,-0.002*x0**2*x1*x2 - 0.007*x0**2*x1*exp((0.003...,535119,0.999991,55,-0.002*x0**2*x1*x2 - 0.007*x0**2*x1*exp((0.003...,False,True,-0.002*x0**2*x1*x2 - 0.007*x0**2*x1*exp((0.003...,305
4,124,2,4,2,3,"[1, 1, 1]","[1, 1, 1]",1,"['sqrt', 'cos', 'exp']",3,...,0.999752,-2.001*x0*cos(1.0*x1*x2 + 6.282) + 2.0*x0,998785,1.000000,4,-2.001*x0*cos(1.0*x1*x2 + 6.282) + 2.0*x0,False,True,x0*(2.0 - 2.001*cos(1.0*x1*x2 + 6.282)),14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,250,2,4,2,3,"[1, 1, 1]","[1, 1, 1]",1,"['sqrt', 'cos', 'exp']",3,...,0.999925,(-0.053*x0*x1**2 + 0.708*x0*x1 - 0.053*x1**2)/...,9732,0.999998,5,(-0.053*x0*x1**2 + 0.708*x0*x1 - 0.053*x1**2)/...,True,True,x1*(-0.053*x0*x1 + 0.708*x0 - 0.053*x1)/(0.709...,20
129,251,2,4,2,3,"[1, 1, 1]","[1, 1, 1]",1,"['sqrt', 'cos', 'exp']",3,...,0.999933,(-0.091*x0**3*x1 + 0.0217060000480996*x0**3*ex...,998048,0.999341,38,(-0.091*x0**3*x1 + 0.0217060000480996*x0**3*ex...,False,True,(-0.091*x0**3*x1 + 0.0217060000480996*x0**3*ex...,163
130,252,2,4,2,3,"[1, 1, 1]","[1, 1, 1]",1,"['sqrt', 'cos', 'exp']",3,...,0.999848,(0.074*x0**3*x1 - 0.118*x0**3*sqrt(Abs((0.059*...,999458,0.998884,44,(0.074*x0**3*x1 - 0.118*x0**3*sqrt(Abs((0.059*...,False,False,(0.074*x0**3*x1 - 0.118*x0**3*sqrt(Abs((0.059*...,181
131,253,2,4,2,3,"[1, 1, 1]","[1, 1, 1]",1,"['sqrt', 'cos', 'exp']",3,...,0.999879,-3.315*x0**3 - 0.052*x0**2*x1 + 0.001*x0**2 + ...,1372,0.999996,13,-3.326*x0**3 - 0.025*x0**2*x1 - 0.005*x0**2 + ...,True,True,-3.326*x0**3 - 0.025*x0**2*x1 - 0.005*x0**2 + ...,34
